In [1]:
import pandas as pd
from pandas.tseries.offsets import BDay 
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta 
import time

#Alpha_Vantage is a stock data provider.  This library makes api calls much easier 
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries  

## First, we'll read in the data output by the scraper and do any cleaning that didn't survive the "to_csv" part of the last notebook.  

In [2]:
df = pd.read_csv('AllHoldings.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390168 entries, 0 to 390167
Data columns (total 12 columns):
manager            390168 non-null object
filing_date        390168 non-null object
report_date        390168 non-null object
sh_name            390168 non-null object
sh_class           390168 non-null object
cusip              390168 non-null object
val1000            390168 non-null int64
share_count        390168 non-null int64
share_or_prin      390168 non-null object
discretion         390168 non-null object
sole_vote_amt      390168 non-null int64
shared_vote_amt    390168 non-null int64
dtypes: int64(4), object(8)
memory usage: 35.7+ MB


In [4]:
df['report_date'] = pd.to_datetime(df.report_date)

In [5]:
df['filing_date'] = pd.to_datetime(df.filing_date)

## The next step is to return only the top 20 holdings, aggregated by date and Cusip/Share.   

In [6]:
# Sums up the holdings by date/cusip and returns a flattened dataframe with report_date as the index
summed_df = df.groupby(['report_date', 'cusip', 'sh_name'])['report_date', 'cusip', 'sh_name', 'val1000'].sum().reset_index()
summed_df.set_index('report_date')

,cusip,sh_name,val1000
report_date,,,
2013-06-30,000360206,AAON INC,1179
2013-06-30,000361105,AAR CORP,2039
2013-06-30,000375204,ABB LTD,4802
2013-06-30,00081T108,ACCO BRANDS CORP,65
2013-06-30,000957100,ABM INDS INC,1731
2013-06-30,00101J106,THE ADT CORPORATION,12867
2013-06-30,001031103,AEP INDS INC,536
2013-06-30,00104Q107,AFC ENTERPRISES INC,4421
2013-06-30,001055102,AFLAC INC,75176


In [7]:
#Sets variables that will be used to loop through each report_date and select the top 20 by notional value
looper = summed_df.report_date.value_counts().index
top_df = pd.DataFrame(columns=['report_date', 'cusip', 'sh_name', 'val1000'])

In [8]:
for x in looper:
    top_df = top_df.append(summed_df[(summed_df.report_date == x)].nlargest(20, 'val1000'))

In [9]:
top_df = top_df.sort_values(['report_date', 'val1000'], ascending=[False, False]).reset_index().drop('index', axis=1)

## Now, the goal is to pull back stock tickers for all of the stocks that appear in our 420 row dataframe.  

#### This is a somewhat complicated task as stocks can undergo certain changes (which is why CUSIPS are often used as an identifier.

#### Fidelity seems to be the only free source of a cusip lookup, so I will first scrape there.  If there aren't too many NaNs after scraping Fidelity, I'll just figure out the rest of the tickers by hand.  If there are a lot, then I'll try some other automated solution.

In [10]:
# creates a list of unique cusips.
cusips = list(top_df.cusip.unique())

In [11]:
cusips[:5]

['037833100', '78462F103', '949746101', '060505104', '500754106']

In [12]:
#cusip_ticker = pd.DataFrame({'cusip':cusips, 'tickers':ticker})
cusip_ticker = pd.read_csv('cusip_ticker.csv')  
cusip_ticker.columns=['d', 'cusip', 'ticker']
cusip_ticker.drop('d', axis=1)
cusip_ticker.head()

,d,cusip,ticker
0,0,949746101,WFC
1,1,500754106,KHC
2,2,191216100,KO
3,3,459200101,IBM
4,4,025816109,AXP


In [13]:
cusip_ticker[cusip_ticker.ticker.str.contains('Not')]
cusip_ticker.iloc[7,2] = 'SPY' 
cusip_ticker.iloc[10,2] = 'SPY' 
cusip_ticker.iloc[14,2] = 'VRX' #Company acquired by Bausch and Lomb 
cusip_ticker.iloc[20,2] = 'AAPL' #Looks like a miskey by the 13F filer
cusip_ticker.iloc[21,2] = 'DTV' #Company acquired by AT&T
cusip_ticker.iloc[29,2] = 'AGN' #Company acquired
cusip_ticker.iloc[33,2] = 'EMC' #Company acquired
cusip_ticker.iloc[46,2] = 'QQQ' #Fidelity tool not able to handle ETFs
cusip_ticker.iloc[48,2] = 'PCLN' #Company acquired by Bausch and Lomb 
cusip_ticker.iloc[56,2] = 'FRX' #Company acquired 
cusip_ticker.iloc[57,2] = 'MDT' #Company acquired 
cusip_ticker.iloc[58,2] = 'DWDP' #Company acquired 
cusip_ticker.iloc[60,2] = 'DELL' #Company went private 
cusip_ticker.iloc[61,2] = 'YHOO' #Company acquired 
cusip_ticker.iloc[64,2] = 'FDML' #Company went private 


## The above "alpha_data" dataframe came from the alphavantage API.  Virtually all of the cusips belonging to the tickers above (cusips not found) were missing from AV.  So, I instead sourced the data from another source that has unadjusted close prices only.  Given the infrequency of my strategy's trading, the likelihood of corporate actions having a significant effect is low.

In [14]:
top_df = pd.merge(top_df, cusip_ticker, how='left', on='cusip').drop('d', axis=1)

##  The dates I have are the dates when the investors held a position.  Their reports are filed 43-45 days after this date.  To make my life easier, I ignored the "filing_date" column from my 13F scraper and just use the next business day 44 days after the "report_date" col. 

In [15]:
top_df['buy_date'] = top_df['report_date'].apply(lambda x: x +  timedelta(days=44) + BDay(1))

In [16]:
top_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 6 columns):
report_date    420 non-null datetime64[ns]
cusip          420 non-null object
sh_name        420 non-null object
val1000        420 non-null object
ticker         420 non-null object
buy_date       420 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(4)
memory usage: 23.0+ KB


In [17]:
buy_prices = pd.read_csv('Yahoo_Prices.csv')
buy_prices.columns


Index(['Date', 'Adj Close', 'ticker'], dtype='object')

In [18]:
# prices = prices.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1 )
buy_prices.Date = pd.to_datetime(buy_prices.Date)
buy_prices.Date = buy_prices.Date.apply(lambda x: x.date())
buy_filtered = pd.DataFrame(columns=['Date', 'Adj Close', 'ticker'])

In [19]:
x = top_df['buy_date'].dt.date.unique()
for z in x:
    buy_filtered = buy_filtered.append(buy_prices[buy_prices.Date==z])

In [20]:
buy_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1071 entries, 30 to 67709
Data columns (total 3 columns):
Date         1071 non-null object
Adj Close    1071 non-null float64
ticker       1071 non-null object
dtypes: float64(1), object(2)
memory usage: 33.5+ KB


In [21]:
buy_filtered.reset_index(inplace=True)

In [22]:
buy_filtered.head()

,index,Date,Adj Close,ticker
0,30,2018-08-14,78.309998,XOM
1,1349,2018-08-14,90.849998,WMT
2,2668,2018-08-14,58.070000,WFC
3,3987,2018-08-14,41.930000,VWO
4,5310,2018-08-14,53.169998,USB


In [23]:
buy_filtered = buy_filtered.drop('index', axis=1)
buy_filtered.head()

,Date,Adj Close,ticker
0,2018-08-14,78.309998,XOM
1,2018-08-14,90.849998,WMT
2,2018-08-14,58.070000,WFC
3,2018-08-14,41.930000,VWO
4,2018-08-14,53.169998,USB


In [24]:
buy_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 3 columns):
Date         1071 non-null object
Adj Close    1071 non-null float64
ticker       1071 non-null object
dtypes: float64(1), object(2)
memory usage: 25.2+ KB


In [25]:
buy_filtered.Date = pd.to_datetime(buy_prices.Date)

In [26]:
top_df.sort_values(['buy_date', 'ticker'], inplace = True, ascending=True)
top_df.reset_index(drop= True, inplace = True)

In [27]:
top_df.head()

,report_date,cusip,sh_name,val1000,ticker,buy_date
0,2013-06-30,025816109,AMERICAN EXPRESS CO,11452604,AXP,2013-08-14
1,2013-06-30,20825C104,CONOCOPHILLIPS,1511450,COP,2013-08-14
2,2013-06-30,13645T100,CANADIAN PAC RY LTD,2957935,CP,2013-08-14
3,2013-06-30,12662P108,CVR ENERGY INC,3416439,CVI,2013-08-14
4,2013-06-30,24702R101,DELL INC,2450000,DELL,2013-08-14


In [28]:
top_df = pd.merge(top_df, buy_filtered, how='left', left_on=['ticker', 'buy_date'], right_on=['ticker', 'Date'])

In [29]:
top_df['sell_date'] = top_df.buy_date.shift(periods = -20)
top_df.drop('Date', axis=1, inplace=True)

In [30]:
top_df = pd.merge(top_df, buy_filtered, how='left', left_on=['ticker', 'sell_date'], right_on=['ticker', 'Date'])

In [31]:
top_df[top_df.ticker=='QQQ']

,report_date,cusip,sh_name,val1000,ticker,buy_date,Adj Close_x,sell_date,Date,Adj Close_y
296,2016-12-31,73935A104,POWERSHARES QQQ TRUST,3755022,QQQ,2017-02-14,NaN,2017-05-15,NaT,NaN
316,2017-03-31,73935A104,POWERSHARES QQQ TRUST,4444607,QQQ,2017-05-15,NaN,2017-08-14,NaT,NaN
336,2017-06-30,73935A104,POWERSHARES QQQ TRUST,4330193,QQQ,2017-08-14,NaN,2017-11-14,NaT,NaN
375,2017-12-31,73935A104,POWERSHARES QQQ TRUST,5140654,QQQ,2018-02-14,NaN,2018-05-15,NaT,NaN
396,2018-03-31,73935A104,POWERSHARES QQQ TRUST,9494668,QQQ,2018-05-15,NaN,2018-08-14,NaT,NaN
415,2018-06-30,46090E103,INVESCO QQQ TR,8792866,QQQ,2018-08-14,NaN,NaT,NaT,NaN


In [76]:
top_df.rename(columns={'Adj Close_x': 'buy_price', 'Adj Close_y': 'sell_price'}, inplace=True)
top_df.drop('Date', axis=1, inplace=True)

In [77]:
top_df[top_df.ticker=='AAPL'].head(7)

,report_date,cusip,sh_name,val1000,ticker,buy_date,buy_price,sell_date,sell_price,buy_price,sell_price
20,2013-09-30,037833100,APPLE INC,2753258,AAPL,2013-11-14,69.1578,2014-02-14,71.657,NaN,NaN
40,2013-12-31,037833100,APPLE INC,6451807,AAPL,2014-02-14,71.657,2014-05-15,77.9954,NaN,NaN
60,2014-03-31,037833100,APPLE INC,7460756,AAPL,2014-05-15,77.9954,2014-08-14,90.854,NaN,NaN
80,2014-06-30,037833100,APPLE INC,9073138,AAPL,2014-08-14,90.854,2014-11-14,106.858,NaN,NaN
100,2014-09-30,037833100,APPLE INC,9466057,AAPL,2014-11-14,106.858,2014-11-14,106.858,NaN,NaN
122,2014-12-31,137833100,APPLE INC,5823743,AAPL,2015-02-16,120.105,2015-02-16,120.105,NaN,NaN
123,2014-12-31,037833100,APPLE INC,4620136,AAPL,2015-02-16,120.105,2015-05-15,121.494,NaN,NaN


In [78]:
portfolio = top_df[['ticker', 'sh_name', 'buy_date', 'buy_price', 'sell_date', 'sell_price']]

In [79]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 0 to 422
Data columns (total 8 columns):
ticker        423 non-null object
sh_name       423 non-null object
buy_date      423 non-null datetime64[ns]
buy_price     423 non-null object
buy_price     5 non-null float64
sell_date     403 non-null datetime64[ns]
sell_price    403 non-null object
sell_price    7 non-null float64
dtypes: datetime64[ns](2), float64(2), object(4)
memory usage: 29.7+ KB


In [36]:
portfolio['shares_bought'] = 1
portfolio['notional_bought'] = 1
portfolio['notional_sold'] = 1
sum_not_sold = 10000000

In [37]:
portfolio.loc[portfolio.buy_date=='2013-08-14','shares_bought'] = sum_not_sold/20/portfolio['buy_price']

TypeError: unsupported operand type(s) for /: 'float' and 'str'

In [160]:
portfolio.loc[portfolio.buy_date=='2013-08-14','notional_bought'] = portfolio['shares_bought']*portfolio['buy_price']

In [161]:
portfolio.loc[portfolio.buy_date=='2013-08-14','notional_sold'] = portfolio['shares_bought']*portfolio['sell_price']

In [162]:
portfolio.head(22)

,ticker,sh_name,buy_date,buy_price,sell_date,sell_price,shares_bought,notional_bought,notional_sold
0,AXP,AMERICAN EXPRESS CO,2013-08-14,70.487648,2013-11-14,76.301682,7093.441393,500000.0,541241.509434
1,COP,CONOCOPHILLIPS,2013-08-14,57.361557,2013-11-14,63.558880,8716.639264,500000.0,554019.828995
2,CP,CANADIAN PAC RY LTD,2013-08-14,115.452370,2013-11-14,140.971481,4330.790264,500000.0,610517.917475
3,CVI,CVR ENERGY INC,2013-08-14,28.293880,2013-11-14,24.337425,17671.666099,500000.0,430082.848305
4,DELL,DELL INC,2013-08-14,13.720000,2013-11-14,13.840000,36443.148688,500000.0,504373.177843
5,DTV,DIRECTV,2013-08-14,61.410000,2013-11-14,64.350000,8141.996418,500000.0,523937.469468
6,DVA,DAVITA HEALTHCARE PARTNERS I,2013-08-14,57.299999,2013-11-14,58.689999,8726.003643,500000.0,512129.145063
7,EEM,ISHARES TR,2013-08-14,36.305775,2013-11-14,37.537094,13771.913697,500000.0,516957.619001
8,IBM,INTERNATIONAL BUSINESS MACHS,2013-08-14,155.742050,2013-11-14,152.136490,3210.436745,500000.0,488424.577691
9,IEP,ICAHN ENTERPRISES LP,2013-08-14,49.667171,2013-11-14,75.680756,10067.011870,500000.0,761879.068973


In [163]:
sum_not_sold = portfolio.loc[portfolio.buy_date=='2013-08-14'].groupby('buy_date')['notional_sold'].sum()

In [164]:
sum_not_sold = sum_not_sold.values

In [166]:
portfolio.loc[portfolio.buy_date=='2013-11-14','shares_bought'] = sum_not_sold/20/portfolio['buy_price']

In [167]:
portfolio.loc[portfolio.buy_date=='2013-11-14','notional_bought'] = portfolio['shares_bought']*portfolio['buy_price']

In [168]:
portfolio.loc[portfolio.buy_date=='2013-11-14','notional_sold'] = portfolio['shares_bought']*portfolio['sell_price']

In [169]:
portfolio.loc[portfolio.buy_date=='2013-11-14']

,ticker,sh_name,buy_date,buy_price,sell_date,sell_price,shares_bought,notional_bought,notional_sold
20,AAPL,APPLE INC,2013-11-14,69.157845,2014-02-14,71.656990,7843.531781,542441.755191,562043.878425
21,APD,AIR PRODS & CHEMS INC,2013-11-14,87.811844,2014-02-14,95.496201,6177.318804,542441.755191,589910.478187
22,AXP,AMERICAN EXPRESS CO,2013-11-14,76.301682,2014-02-14,83.140488,7109.171659,542441.755191,591060.000986
23,CP,CANADIAN PAC RY LTD,2013-11-14,140.971481,2014-02-14,147.010574,3847.882929,542441.755191,565679.478051
24,CVI,CVR ENERGY INC,2013-11-14,24.337425,2014-02-14,24.343891,22288.379119,542441.755191,542585.871851
25,DTV,DIRECTV,2013-11-14,64.350000,2014-02-14,71.890000,8429.553305,542441.755191,606000.587113
26,EEM,ISHARES,2013-11-14,37.537094,2014-02-14,36.231758,14450.819107,542441.755191,523578.580782
27,FDML,FEDERAL MOGUL CORP,2013-11-14,20.080000,2014-02-14,17.180000,27014.031633,542441.755191,464101.063455
28,GM,GENERAL MTRS CO,2013-11-14,31.392313,2014-02-14,29.313080,17279.445296,542441.755191,506513.762311
29,IBM,INTERNATIONAL BUSINESS MACHS,2013-11-14,152.136490,2014-02-14,154.213013,3565.494085,542441.755191,549845.585665


In [170]:
portfolio.head(42)

,ticker,sh_name,buy_date,buy_price,sell_date,sell_price,shares_bought,notional_bought,notional_sold
0,AXP,AMERICAN EXPRESS CO,2013-08-14,70.487648,2013-11-14,76.301682,7093.441393,500000.000000,541241.509434
1,COP,CONOCOPHILLIPS,2013-08-14,57.361557,2013-11-14,63.558880,8716.639264,500000.000000,554019.828995
2,CP,CANADIAN PAC RY LTD,2013-08-14,115.452370,2013-11-14,140.971481,4330.790264,500000.000000,610517.917475
3,CVI,CVR ENERGY INC,2013-08-14,28.293880,2013-11-14,24.337425,17671.666099,500000.000000,430082.848305
4,DELL,DELL INC,2013-08-14,13.720000,2013-11-14,13.840000,36443.148688,500000.000000,504373.177843
5,DTV,DIRECTV,2013-08-14,61.410000,2013-11-14,64.350000,8141.996418,500000.000000,523937.469468
6,DVA,DAVITA HEALTHCARE PARTNERS I,2013-08-14,57.299999,2013-11-14,58.689999,8726.003643,500000.000000,512129.145063
7,EEM,ISHARES TR,2013-08-14,36.305775,2013-11-14,37.537094,13771.913697,500000.000000,516957.619001
8,IBM,INTERNATIONAL BUSINESS MACHS,2013-08-14,155.742050,2013-11-14,152.136490,3210.436745,500000.000000,488424.577691
9,IEP,ICAHN ENTERPRISES LP,2013-08-14,49.667171,2013-11-14,75.680756,10067.011870,500000.000000,761879.068973


In [171]:
looper = portfolio.buy_date.value_counts().index.sort_values()
looper.date

array([datetime.date(2013, 8, 14), datetime.date(2013, 11, 14),
       datetime.date(2014, 2, 14), datetime.date(2014, 5, 15),
       datetime.date(2014, 8, 14), datetime.date(2014, 11, 14),
       datetime.date(2015, 2, 16), datetime.date(2015, 5, 15),
       datetime.date(2015, 8, 14), datetime.date(2015, 11, 16),
       datetime.date(2016, 2, 15), datetime.date(2016, 5, 16),
       datetime.date(2016, 8, 15), datetime.date(2016, 11, 14),
       datetime.date(2017, 2, 14), datetime.date(2017, 5, 15),
       datetime.date(2017, 8, 14), datetime.date(2017, 11, 14),
       datetime.date(2018, 2, 14), datetime.date(2018, 5, 15),
       datetime.date(2018, 8, 14)], dtype=object)

In [172]:
for date in looper:
    portfolio.loc[portfolio.buy_date==pd.to_datetime(date),'shares_bought'] = sum_not_sold/20/portfolio['buy_price']
    portfolio.loc[portfolio.buy_date==pd.to_datetime(date),'notional_bought'] = portfolio['shares_bought']*portfolio['buy_price']
    portfolio.loc[portfolio.buy_date==pd.to_datetime(date),'notional_sold'] = portfolio['shares_bought']*portfolio['sell_price']
    sum_not_sold = portfolio.loc[portfolio.buy_date==pd.to_datetime(date)].groupby('buy_date')['notional_sold'].sum()
    sum_not_sold = sum_not_sold.values

In [177]:
portfolio[portfolio.notional_sold.isna()]

,ticker,sh_name,buy_date,buy_price,sell_date,sell_price,shares_bought,notional_bought,notional_sold
106,DWDP,DOW CHEM CO,2014-11-14,45.429230,2015-02-16,NaN,14911.724324,677428.154023,NaN
107,EEM,ISHARES TR,2014-11-14,38.157028,2015-02-16,NaN,17753.692820,677428.154023,NaN
120,XOM,EXXON MOBIL CORP,2014-11-14,82.971992,2015-02-16,NaN,8164.540078,677428.154023,NaN
167,DTV,DIRECTV,2015-08-14,93.550000,2015-11-16,NaN,6960.031531,651110.949700,NaN
248,EMC,E M C CORP MASS,2016-08-15,28.470000,2016-11-14,NaN,22409.821500,638007.618091,NaN
357,PCLN,PRICELINE GRP INC,2017-11-14,1719.500000,2018-02-14,NaN,431.751997,742397.559087,NaN
402,AAPL,APPLE INC,2018-08-14,209.750000,NaT,NaN,3841.824140,805822.613453,NaN
403,AMZN,AMAZON COM INC,2018-08-14,1919.650024,NaT,NaN,419.775794,805822.613453,NaN
404,AXP,AMERICAN EXPRESS CO,2018-08-14,102.180000,NaT,NaN,7886.304692,805822.613453,NaN
405,BABA,ALIBABA GROUP HLDG LTD,2018-08-14,172.529999,NaT,NaN,4670.623185,805822.613453,NaN


In [134]:
for date in looper:
    portfolio.loc[portfolio.buy_date==pd.to_datetime(date),'shares_bought'] = sum_not_sold/20/portfolio['buy_price']
    portfolio.loc[portfolio.buy_date==pd.to_datetime(date),'notional_bought'] = portfolio['shares_bought']*portfolio['buy_price']
    portfolio.loc[portfolio.buy_date==pd.to_datetime(date),'notional_sold'] = portfolio['shares_bought']*portfolio['sell_price']
    sum_not_sold = portfolio.loc[portfolio.buy_date==pd.to_datetime(date)].groupby('buy_date')['notional_sold'].sum()
    sum_not_sold = sum_not_sold.values

In [135]:
portfolio

,ticker,sh_name,buy_date,buy_price,sell_date,sell_price,shares_bought,notional_bought,notional_sold
0,AXP,AMERICAN EXPRESS CO,2013-08-14,70.487648,2013-11-14,76.301682,0.0,0.0,0.0
1,COP,CONOCOPHILLIPS,2013-08-14,57.361557,2013-11-14,63.558880,0.0,0.0,0.0
2,CP,CANADIAN PAC RY LTD,2013-08-14,115.452370,2013-11-14,140.971481,0.0,0.0,0.0
3,CVI,CVR ENERGY INC,2013-08-14,28.293880,2013-11-14,24.337425,0.0,0.0,0.0
4,DELL,DELL INC,2013-08-14,13.720000,2013-11-14,13.840000,0.0,0.0,0.0
5,DTV,DIRECTV,2013-08-14,61.410000,2013-11-14,64.350000,0.0,0.0,0.0
6,DVA,DAVITA HEALTHCARE PARTNERS I,2013-08-14,57.299999,2013-11-14,58.689999,0.0,0.0,0.0
7,EEM,ISHARES TR,2013-08-14,36.305775,2013-11-14,37.537094,0.0,0.0,0.0
8,IBM,INTERNATIONAL BUSINESS MACHS,2013-08-14,155.742050,2013-11-14,152.136490,0.0,0.0,0.0
9,IEP,ICAHN ENTERPRISES LP,2013-08-14,49.667171,2013-11-14,75.680756,0.0,0.0,0.0


DatetimeIndex(['2014-11-14', '2015-02-16', '2013-08-14', '2015-11-16',
               '2015-05-15', '2018-05-15', '2016-02-15', '2013-11-14',
               '2016-11-14', '2014-08-14', '2017-08-14', '2016-08-15',
               '2016-05-16', '2014-02-14', '2017-02-14', '2014-05-15',
               '2017-05-15', '2017-11-14', '2015-08-14', '2018-08-14',
               '2018-02-14'],
              dtype='datetime64[ns]', freq=None)